In [49]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from AP_class import APDFT_perturbator as AP
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [90]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP
from ABSE import absec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from berny import Berny, geomlib, coords,Geometry,Math

In [3]:
HF=gto.M(atom="F 0 0 0;H 0 0 1.695741",unit="bohr",
            basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mfhf = scf.RHF(HF)
mfhf.scf()

converged SCF energy = -100.065540168732


-100.06554016873231

In [4]:
dm=mfhf.make_rdm1()

In [5]:
def Vne(coords,mol=HF,dm=dm):
    mol.set_rinv_orig_(coords)
    dV=mol.intor('int1e_rinv')
    return -np.einsum('ij,ij',dV,dm)

In [6]:
dL=[-1,0,0,0] #-1 charge central atom
def Vnn(coords,mol=HF):
    Vnn=0
    for i in range (mol.natm):
        Vnn+= (mol.atom_charge(i)+dL[i])/np.linalg.norm(coords-mol.atom_coord(i))
    return Vnn

In [7]:
Vnn([0,.5,-1.3],HF)

6.072918030765468

In [8]:
def Vtot(coords,mol=HF,dm=dm):
    return Vnn(coords,mol)+Vne(coords,mol,dm)

In [9]:
def Vtot_fl(angle,mol=HF,dm=dm):
    coords=np.asarray([0,np.sin(angle),np.cos(angle)])*1.695741
    return Vtot(coords,mol=HF,dm=dm)

In [10]:
prot_angle=minimize(Vtot_fl,1.8).x[0]
prot_angle*180/np.pi

119.57798490677513

In [11]:
prot_angle

2.0870295495233107

In [12]:
prot_site=np.array([0.,np.sin(prot_angle),np.cos(prot_angle)])*1.695741
prot_site

array([ 0.        ,  1.47475993, -0.83703088])

In [13]:
prot_site=minimize(Vtot,[0,.5,-1.3]).x
norm(prot_site),np.arccos(prot_site[2]/norm(prot_site))*180/np.pi

(1.5510757176935956, 118.42150068174293)

In [14]:
def dVne(coords,mol=HF,dm=dm):
    mol.set_rinv_orig_(coords)
    vrinv = -mol.intor('int1e_iprinv', comp=3)
    vrinv += vrinv.transpose(0,2,1)
    return np.einsum('xij,ij->x',vrinv,dm)

In [15]:
dVne([0,.5,-1.3])

array([ 7.46991946e-14,  1.38257692e+00, -3.72835980e+00])

In [16]:
Vne([0,.5,-1.3],HF,dm)-Vne([0.,.5,-1.301],HF,dm)

-0.003726764702753016

In [17]:
#fig=plt.figure(figsize=(20, 10))
##ax = plt.axes(projection=’3d’)
#ax =fig.add_subplot(111, projection='3d')
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1,cmap="terrain" ,edgecolor=None)
#ax.set(xlabel='x', ylabel='y', zlabel='Energy',title="Probing PES for protonation")

In [18]:
#from matplotlib import cm
#colormap = cm.coolwarm
#znorm = Z - Z.min()
#znorm /= znorm.ptp()
#znorm.min(), znorm.max()
#color = colormap(znorm)

In [19]:
#ipv.figure()
#mesh = ipv.plot_surface(X, Y, 100*Z, color=color[...,:3])
#ipv.show()

In [20]:
prot_site

array([-7.15031068e-07,  1.36412461e+00, -7.38241112e-01])

# How to develop the basis functions or do none.


In [21]:
pHF=FcM(fcs=[0,0,-1],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1)
pHF.charge=0

In [22]:
pHF.atom_charges(),pHF.nelec

(array([9., 1., 0.]), (5, 5))

In [23]:
pHFmf=scf.RHF(pHF)

In [92]:
pHFmf.mo_occ=np.zeros(pHF.nao)
pHFmf.mo_occ[:HF.nao]=mfhf.mo_occ
pHFmf.mo_energy=np.zeros(pHF.nao)
pHFmf.mo_energy[:HF.nao]=mfhf.mo_energy
pHFmf.mo_coeff=np.zeros([pHF.nao,pHF.nao])
pHFmf.mo_coeff[:HF.nao,:HF.nao]=mfhf.mo_coeff
pHFmf.e_tot=pHFmf.energy_tot()+absec("F","O")

In [93]:
pHFmf.make_rdm1()[:HF.nao,:HF.nao]-dm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [94]:
pHFmf.e_tot

-100.0685069326202

In [95]:
af=aaff(pHFmf,[-1,0,1])+alc_deriv_grad_nuc(pHF,[-1,0,1])

In [96]:
apHF=AP(pHFmf,sites=[0,2])
apHF.build_all()

In [97]:
apHF.APDFT3([-1,1])

-75.89809296805801

In [98]:
phfc=pHF.atom_coords()*bohr2ang
pbg=Geometry(['O','H','H'],phfc)
b=Berny(pbg,debug=True)

In [99]:
b.send((apHF.APDFT3([-1,1]),af))

{'geom': <Geometry 'H2O'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 2, angles: 1, dihedrals: 0">,
 'H': array([[0.54502214, 0.        , 0.        ],
        [0.        , 0.58393729, 0.        ],
        [0.        , 0.        , 0.23574722]]),
 'weights': array([1.21116031, 1.29763842, 1.1206834 ]),
 'future': Point(q=array([1.72184677, 1.84993775, 2.06685442]), E=None, g=None),
 'first': False,
 'interpolated': Point(q=array([1.695741  , 1.55107572, 2.06684509]), E=-75.89809296805801, g=array([-3.52924845e-02, -4.15661747e-01, -9.92128299e-06])),
 'predicted': Point(q=array([1.72184684, 1.8499377 , 2.06685461]), E=-75.99697588463177, g=None),
 'previous': Point(q=array([1.695741  , 1.55107572, 2.06684509]), E=-75.89809296805801, g=array([-3.52924845e-02, -4.15661747e-01, -9.92128299e-06])),
 'best': Point(q=array(

In [112]:
2.06685*180/np.pi

118.4217818866142

In [30]:
q=np.array([1.72184679, 1.84993778, 2.06685607])  # bd H1_O,bd H2_O, angle (H-O-H) -- Bohr,radians

In [31]:
norm(phfc[1])*ang2bohr,norm(phfc[2])*ang2bohr,np.arccos((phfc[1].dot(phfc[2])/norm(phfc[1])/norm(phfc[2])))

(1.6957409999672617, 1.5510757176636505, 2.0668450920491237)

In [32]:
pbg.coords,phfc

(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  8.97347493e-01],
        [-3.78378146e-07,  7.21863655e-01, -3.90660372e-01]]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  8.97347493e-01],
        [-3.78378146e-07,  7.21863655e-01, -3.90660372e-01]]))

# H$_2$O

In [33]:
OH2=gto.M(atom="O 0 0 0;H 0 -0.15229 1.76907; H 0   1.7401 -0.35319",unit="bohr",
            basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mfoh2 = scf.RHF(OH2)
mfoh2.scf()
dmoh2=mfoh2.make_rdm1()

converged SCF energy = -76.0633193329876


In [34]:
ps_h2o=minimize(Vtot,np.array([1.5,0,0]),args=(OH2,dmoh2)).x

In [35]:
pH2O=FcM(fcs=[0,0,0,-1],atom="O 0 0 0;H 0 -0.15229 1.76907; H 0 1.7401 -0.35319; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2O.charge=0

In [101]:
pH2Omf=scf.RHF(pH2O)
pH2Omf.mo_occ=np.zeros(pH2O.nao)
pH2Omf.mo_occ[:OH2.nao]=mfoh2.mo_occ
pH2Omf.mo_energy=np.zeros(pH2O.nao)
pH2Omf.mo_energy[:OH2.nao]=mfoh2.mo_energy
pH2Omf.mo_coeff=np.zeros([pH2O.nao,pH2O.nao])
pH2Omf.mo_coeff[:OH2.nao,:OH2.nao]=mfoh2.mo_coeff
pH2Omf.e_tot=pH2Omf.energy_tot()+absec("O","N")

In [37]:
afph2o=aaff(pH2Omf,[-1,0,0,1])+alc_deriv_grad_nuc(pH2O,[-1,0,0,1])

In [102]:
aph2o=AP(pH2Omf,sites=[0,3])
aph2o.build_all()

In [103]:
ph2oc=pH2O.atom_coords()*bohr2ang
pbg2=Geometry(['N','H','H','H'],ph2oc)
b2=Berny(pbg2,debug=True)
b2.send((aph2o.APDFT3([-1,1]),afph2o))

{'geom': <Geometry 'H3N'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 3, angles: 3, dihedrals: 0">,
 'H': array([[0.53257882, 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.53258653, 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.57335339, 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.21010689, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.22618953,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.22619281]]),
 'weights': array([1.18350849, 1.18352562, 1.27411863, 1.14117516, 1.09421473,
        1.0942179 ]),
 'future': Point(q=array([1.81528433, 1.81525516, 1.91249063, 1.87018281,

In [113]:
1.87018279*180/np.pi, 2.07363962*180/np.pi

(107.15358078500114, 118.8107984571118)

#     NH$_3$

In [39]:
ch4optg,nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy")
nh3lst=[]
for x in range(4):
    nh3lst.append("NHHH"[x])
    nh3lst.extend(nh3optg[x].tolist()) 
nh3srt=(("{} "*4+";")*4)[:-1].format(*nh3lst)
NH3=gto.M(atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf3 = scf.RHF(NH3)
mf3.scf()

converged SCF energy = -56.2221836146945


-56.222183614694465

In [40]:
nh3optg

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.90982817e-13,  3.71707821e-01,  1.84851805e+00],
       [ 1.52736243e+00,  8.01399691e-01, -7.61648360e-01],
       [-1.52736243e+00,  8.01399691e-01, -7.61648360e-01]])

In [41]:
nh3dm=mf3.make_rdm1()
ps_nh3=minimize(Vtot,np.array([0,-1.5,.3]),args=(NH3,nh3dm)).x

In [42]:
pNH3=FcM(fcs=[0,0,0,0,-1],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3.charge=0

In [105]:
pNH3mf=scf.RHF(pNH3)
pNH3mf.mo_occ=np.zeros(pNH3.nao)
pNH3mf.mo_occ[:NH3.nao]=mf3.mo_occ
pNH3mf.mo_energy=np.zeros(pNH3.nao)
pNH3mf.mo_energy[:NH3.nao]=mf3.mo_energy
pNH3mf.mo_coeff=np.zeros([pNH3.nao,pNH3.nao])
pNH3mf.mo_coeff[:NH3.nao,:NH3.nao]=mf3.mo_coeff
pNH3mf.e_tot=pNH3mf.energy_tot()+absec("N","C")

In [44]:
afpnh3=aaff(pNH3mf,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3,[-1,0,0,0,1])

In [107]:
apNH3=AP(pNH3mf,sites=[0,4])
apNH3.build_all()

In [109]:
pnh3c=pNH3.atom_coords()*bohr2ang
pbg3=Geometry(['C','H','H','H','H'],pnh3c)
b3=Berny(pbg3,debug=True)
b3.send((apNH3.APDFT3([-1,1]),afpnh3))

{'geom': <Geometry 'CH4'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 4, angles: 6, dihedrals: 0">,
 'H': array([[0.5136889 , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.5136889 , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.5136889 , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.55378222, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.19546392,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.    

In [114]:
1.8809583*180/np.pi , 1.93958*180/np.pi

(107.77097203010216, 111.12974802798422)

In [110]:
b3._state.coords._coords

[Bond(0, 1, weak=0),
 Bond(0, 2, weak=0),
 Bond(0, 3, weak=0),
 Bond(0, 4, weak=0),
 Angle(1, 0, 2, weak=0),
 Angle(1, 0, 3, weak=0),
 Angle(1, 0, 4, weak=0),
 Angle(2, 0, 3, weak=0),
 Angle(2, 0, 4, weak=0),
 Angle(3, 0, 4, weak=0)]

In [111]:
pbg4=Geometry(['C','H','H','H','H'],ch4optg*bohr2ang)
Berny(pbg4)._state.future

Point(q=array([2.04385424, 2.04385424, 2.04385424, 2.04385424, 1.91063324,
       1.91063324, 1.91063324, 1.91063324, 1.91063324, 1.91063324]), E=None, g=None)